Kako biste pokrenuli sljedeće bilježnice, ako to još niste učinili, trebate implementirati model koji koristi `text-embedding-ada-002` kao osnovni model i postaviti naziv implementacije u .env datoteku kao `AZURE_OPENAI_EMBEDDINGS_ENDPOINT`


In [ ]:
import os
import pandas as pd
import numpy as np
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv()

client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  # this is also the default, it can be omitted
  api_version = "2023-05-15"
  )

model = os.environ['AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT']

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

Zatim ćemo učitati indeks ugradnji u Pandas DataFrame. Indeks ugradnji pohranjen je u JSON datoteci pod nazivom `embedding_index_3m.json`. Indeks ugradnji sadrži ugradnje za svaki od YouTube transkripata do kraja listopada 2023.


In [ ]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

Sljedeće ćemo napraviti funkciju pod nazivom `get_videos` koja će pretražiti Embedding Index prema upitu. Funkcija će vratiti 5 videa koji su najsličniji upitu. Funkcija radi na sljedeći način:

1. Prvo se stvara kopija Embedding Indexa.
2. Zatim se izračunava Embedding za upit pomoću OpenAI Embedding API-ja.
3. Nakon toga se u Embedding Index dodaje novi stupac pod nazivom `similarity`. Stupac `similarity` sadrži kosinusnu sličnost između Embeddinga upita i Embeddinga za svaki video segment.
4. Sljedeće, Embedding Index se filtrira prema stupcu `similarity`. Embedding Index se filtrira tako da uključuje samo videe koji imaju kosinusnu sličnost veću ili jednaku 0.75.
5. Na kraju, Embedding Index se sortira prema stupcu `similarity` i vraća se 5 najrelevantnijih videa.


In [ ]:
def cosine_similarity(a, b):
    if len(a) > len(b):
        b = np.pad(b, (0, len(a) - len(b)), 'constant')
    elif len(b) > len(a):
        a = np.pad(a, (0, len(b) - len(a)), 'constant')
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

Ova funkcija je vrlo jednostavna, samo ispisuje rezultate pretrage.


In [ ]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

1. Prvo se Embedding Index učitava u Pandas Dataframe.
2. Zatim se od korisnika traži da unese upit.
3. Nakon toga poziva se funkcija `get_videos` kako bi se pretražio Embedding Index prema upitu.
4. Na kraju se poziva funkcija `display_results` koja prikazuje rezultate korisniku.
5. Korisnik se zatim ponovno poziva da unese novi upit. Ovaj postupak se ponavlja dok korisnik ne unese `exit`.

![](../../../../translated_images/notebook-search.1e320b9c7fcbb0bc1436d98ea6ee73b4b54ca47990a1c952b340a2cadf8ac1ca.hr.png)

Bit ćete upitani da unesete upit. Unesite upit i pritisnite enter. Aplikacija će prikazati popis videa koji su povezani s vašim upitom. Također, aplikacija će prikazati poveznicu na dio videa gdje se nalazi odgovor na vaše pitanje.

Evo nekoliko upita koje možete isprobati:

- Što je Azure Machine Learning?
- Kako rade konvolucijske neuronske mreže?
- Što je neuronska mreža?
- Mogu li koristiti Jupyter Notebooks s Azure Machine Learning?
- Što je ONNX?


In [ ]:
pd_vectors = load_dataset(DATASET_NAME)

# get user query from imput
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)


---

**Odricanje od odgovornosti**:  
Ovaj dokument je preveden pomoću AI usluge prevođenja [Co-op Translator](https://github.com/Azure/co-op-translator). Iako nastojimo osigurati točnost, imajte na umu da automatski prijevodi mogu sadržavati pogreške ili netočnosti. Izvorni dokument na izvornom jeziku treba smatrati mjerodavnim izvorom. Za ključne informacije preporučuje se profesionalni ljudski prijevod. Ne snosimo odgovornost za bilo kakva nesporazume ili pogrešna tumačenja koja proizlaze iz korištenja ovog prijevoda.
